## Readme
FastLZ needs Python version 3.9 or lower, make sure your environment is using a later python version

In [1]:
import pandas as pd
from web3 import Web3
from hexbytes import HexBytes
import rlp
from rlp.sedes import Binary, big_endian_int, binary, List
import fastlz
import sys
import os
import dotenv
dotenv.load_dotenv()
sys.path.append("../../helper_functions")
import clickhouse_utils as ch
sys.path.pop()

client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

In [2]:
# Test transaction receipt
from web3 import Web3
op_rpc = os.getenv("OP_PUBLIC_RPC")
w3 = Web3(Web3.HTTPProvider(op_rpc))

tx_test = '0xffae0766dd7a4c6488c57af70348e857953fcad1a49978f1f615511a316bb958'
tx = w3.eth.get_transaction(tx_test)
txr = w3.eth.get_transaction_receipt(tx_test)
print(tx)
# print(txr)

AttributeDict({'accessList': [], 'blockHash': HexBytes('0x636d21f11d6b7319060b800bf95ce36ff09098820ac0c02c1b797a40e5044b10'), 'blockNumber': 120789402, 'chainId': 10, 'from': '0x5592D1f30910c92ad3585AC900E0E7cD40E91e7d', 'gas': 139369, 'gasPrice': 2991508, 'hash': HexBytes('0xffae0766dd7a4c6488c57af70348e857953fcad1a49978f1f615511a316bb958'), 'input': HexBytes('0x2646478b000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee0000000000000000000000000000000000000000000000000000026b12ca3d0000000000000000000000000094b008aa00579c1307b0ef2c499ad98a8ce58e5800000000000000000000000000000000000000000000000000000000000026ed0000000000000000000000005592d1f30910c92ad3585ac900e0e7cd40e91e7d00000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000700301ffff02012f629782c58918252970ab038d3fbd16e84d50764200000000000000000000000000000000000006044200000000000000000000000000000000000006002f629782c58918252970ab038d3fbd16e84d507601

In [3]:
#Configuration
chain_mappings_list = [
        {'schema_name': 'op', 'display_name': 'OP Mainnet', 'chain_id': 10},
        # {'schema_name': 'base', 'display_name': 'Base'},
        # {'schema_name': 'mode', 'display_name': 'Mode'},
        # {'schema_name': 'fraxtal', 'display_name': 'Fraxtal'},
    # Add more mappings as needed
]

days_of_data = 7
# chain_mappings_dict = {item['schema_name']: item['display_name'] for item in chain_mappings_list}


In [4]:
# may not sufficent due to missing transaction signature fields

# Get L2 Txs from Clickhouse / Goldsky
query = '''
        SELECT @chain_id@ as chainId, nonce, gas, max_fee_per_gas, max_priority_fee_per_gas,
                to_address as to, value, input as data, block_number, hash, input, receipt_gas_used
        FROM @chain_db_name@_transactions
        WHERE gas_price > 0
        AND block_timestamp < DATE_TRUNC('day',NOW())
        AND block_timestamp >= DATE_TRUNC('day',NOW() - interval '@num_days@ days')
        LIMIT 1000
'''
dfs = []
for chain in chain_mappings_list:
        query_map = query

        query_map = query_map.replace("@chain_db_name@", chain['schema_name'])
        query_map = query_map.replace("@chain_id@", str(chain['chain_id']))
        query_map = query_map.replace("@num_days@", str(days_of_data))

        result_df = client.query_df(query_map)
        dfs.append(result_df)

txs_df = pd.concat(dfs)

# Add Dummy Signature and fields
txs_df['access_list'] = '[]'
txs_df['r'] = '0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638e367063df71f84991bc9'
txs_df['s'] = '0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fed6c054d4e8d09c0a8e99'
txs_df['v'] = '0'

txs_df

,chainId,nonce,gas,max_fee_per_gas,max_priority_fee_per_gas,to,value,data,block_number,hash,input,receipt_gas_used,access_list,r,s,v
0,10,1392420,150000,10000000000,100000000,b'0x4200000000000000000000000000000000000042',0,0xa9059cbb00000000000000000000000037e98527ebb3...,120530261,b'0xf6a72c719b1fca09d382bbcce70cfd0f72456c8edc...,0xa9059cbb00000000000000000000000037e98527ebb3...,57100,[],0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638...,0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fe...,0
1,10,14,47100,181818308,470902,b'0x94b008aa00579c1307b0ef2c499ad98a8ce58e58',0,0x095ea7b30000000000000000000000006f26bf09b1c7...,120530261,b'0xa7b5c7993abcea525579006809e9a58b7c1a880a5c...,0x095ea7b30000000000000000000000006f26bf09b1c7...,46717,[],0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638...,0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fe...,0
2,10,3,220046,180996613,421216,b'0xc110e7faa95680c79937ccaca3d1cab7902be25e',6881892679325776,0x6b634a62000000000000000000000000000000000000...,120530261,b'0xd4f43e13e1d5a88e49de88d358ec6c338f2b6e4ec1...,0x6b634a62000000000000000000000000000000000000...,211807,[],0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638...,0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fe...,0
3,10,1127963,168391,119895631,373895,b'0x3638d5b2e044e4d1724bf9e77aa59a446ff37005',0,0x6a761202000000000000000000000000b5fbfeba9848...,120530261,b'0xbc916e34e834a06124a0ca41cc7c1ba6e83a9cded5...,0x6a761202000000000000000000000000b5fbfeba9848...,141799,[],0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638...,0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fe...,0
4,10,121958,1423901,119895631,373895,b'0x087000a300de7200382b55d40045000000e5d60e',0,0x82ad56cb000000000000000000000000000000000000...,120530261,b'0xf32565294f872bed10cd3dc7bc3da1c386a62c0581...,0x82ad56cb000000000000000000000000000000000000...,1203819,[],0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638...,0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fe...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10,18,769579,179765128,466870,b'0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae',1690000000000000000,0x4630a0d8d4c2c16d2ac194705b27b4186078f45f5898...,120567248,b'0xdcd6b2e976e499f4a1e18c30af65b4e4845f865b9f...,0x4630a0d8d4c2c16d2ac194705b27b4186078f45f5898...,139689,[],0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638...,0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fe...,0
996,10,243630,533246,0,0,b'0x38edf0a0d32b5c8e23284da87d078062a3f772a7',0,0x2380252cbdff917169775be2b552ec9f6781af95e7f6...,120567248,b'0x68bc48b1272364a21e18ce878386e25ecb478aaed8...,0x2380252cbdff917169775be2b552ec9f6781af95e7f6...,31408,[],0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638...,0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fe...,0
997,10,38779,600000,231154050,100000000,b'0x802b65b5d9016621e66003aed0b16615093f328b',0,0xa00597a0000000000000000000000000000000000000...,120567248,b'0x680b40bff69e91f2d6237bda3375fcbf139bcada53...,0xa00597a0000000000000000000000000000000000000...,59066,[],0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638...,0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fe...,0
998,10,544504,3000000,0,0,b'0xeb8bae7fccc9028269177fb68e6d5c958ca59ac5',0,0xe129b4b8031e01000002271005fa06d4fb883f67f1cf...,120567248,b'0x3d6bb45ee28539de3da1b8fa9cb32c88f95a42cbb1...,0xe129b4b8031e01000002271005fa06d4fb883f67f1cf...,635650,[],0xbcf25be5d635fb2b9fbb3da90e3581357e705bdea638...,0x1002b00f8b701b303a2b517b511d98a18a9e4000d3fe...,0


In [6]:
# Process transactions and RLP encode
def process_and_encode_transaction(tx):
    tx_params = {
        'chainId': int(tx['chainId']),
        'nonce': int(tx['nonce']),
        'gas': int(tx['gas']),
        'maxFeePerGas': int(tx['max_fee_per_gas']),
        'maxPriorityFeePerGas': int(tx['max_priority_fee_per_gas']),
        'to': tx['to'],
        'value': int(tx['value']),
        'data': HexBytes(tx['data']),
        'accessList': [] if pd.isna(tx['access_list']) else eval(tx['access_list']),  # Assuming access_list is JSON stored as string
        'v': int(tx['v']),
        'r': HexBytes(tx['r']),
        's': HexBytes(tx['s'])
    }
    # print(tx_params)

    encoded_tx = rlp.encode([
        tx_params['chainId'],
        tx_params['nonce'],
        tx_params['gas'],
        tx_params['maxFeePerGas'],
        tx_params['maxPriorityFeePerGas'],
        tx_params['to'],
        tx_params['value'],
        tx_params['data'],
        tx_params['accessList'],
        tx_params['v'],
        tx_params['r'],
        tx_params['s']
    ])
    # encoded_signed_tx = encode(signed_tx)
    return Web3.to_hex(encoded_tx)

txs_df['encoded_transaction'] = txs_df.apply(process_and_encode_transaction, axis=1)
txs_df['len_encoded_transaction'] = txs_df['encoded_transaction'].str.len()


In [12]:
! python --version
! gcc --version

Python 3.10.11
Apple clang version 15.0.0 (clang-1500.3.9.4)
Target: x86_64-apple-darwin23.4.0
Thread model: posix
InstalledDir: /Library/Developer/CommandLineTools/usr/bin


In [9]:
# Generate modeled metrics (rlp size, fastlz)
def flz_compress_len(ib):
    n = 0
    ht = [0] * 8192

    def u24(i):
        return ib[i] | (ib[i + 1] << 8) | (ib[i + 2] << 16)

    def cmp(p, q, e):
        l = 0
        for _ in range(e - q):
            if ib[p + l] != ib[q + l]:
                e = 0
            l += 1
        return l

    def literals(r):
        nonlocal n
        n += 0x21 * (r // 0x20)
        r %= 0x20
        if r != 0:
            n += r + 1

    def match(l):
        nonlocal n
        l -= 1
        n += 3 * (l // 262)
        if l % 262 >= 6:
            n += 3
        else:
            n += 2

    def hash(v):
        return ((2654435769 * v) >> 19) & 0x1fff

    def set_next_hash(ip):
        ht[hash(u24(ip))] = ip
        return ip + 1

    a = 0
    ip_limit = len(ib) - 13
    if len(ib) < 13:
        ip_limit = 0

    ip = a + 2
    while ip < ip_limit:
        r = 0
        d = 0
        while True:
            s = u24(ip)
            h = hash(s)
            r = ht[h]
            ht[h] = ip
            d = ip - r
            if ip >= ip_limit:
                break
            ip += 1
            if d <= 0x1fff and s == u24(r):
                break
        if ip >= ip_limit:
            break
        ip -= 1
        if ip > a:
            literals(ip - a)
        l = cmp(r + 3, ip + 3, ip_limit + 9)
        match(l)
        ip = set_next_hash(set_next_hash(ip + l))
        a = ip

    literals(len(ib) - a)
    return n
# Function to compress transaction data
def compress_transaction(encoded_transaction):
    compressed_data = fastlz.compress(encoded_transaction.encode('utf-8'))
    return compressed_data
# Apply compression to each transaction in the DataFrame
txs_df['compressed_transaction'] = txs_df['encoded_transaction'].apply(compress_transaction)


SystemError: PY_SSIZE_T_CLEAN macro must be defined for '#' formats

In [ ]:
# Aggregate L2

In [ ]:
# Pull aggregate L1 data

In [ ]:
# Generate L2 : L1 ratio metrics